In [1]:
# importamos las librerías con las que vamos a trabajar
# !pip install psycopg2==2.9.9
# !pip install psycopg2-binary==2.9.9
# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd

In [2]:
pd.options.display.max_columns = None

In [3]:
df_4sq = pd.read_csv("../../Datos/api_foursquare.csv", index_col= 0)
df_aemet = pd.read_csv("../../Datos/df_aemet_final.csv")
df_municipios = pd.read_csv("../../Datos/municipios.csv",index_col = 0)

Plantearme 5 preguntas con estos datos:

-Eres una empresa de grabación que quiere grabar una película en un parque en los alreedores de Madrid, cuál es el sitio con el mejor tiempo en gral?

-En qué municipio hace más sol para grabar


-Dónde puedo encontrar un castillo para grabar un episodio medieval? Necesito que haya un estudio de películas cerca.


-Dónde hay más probabilidad de que nieve?


In [4]:
#Análisis exploratorio
##df_4sq tiene información de lugares de interés en los diferentes municipios, sus horarios, nombre, dirección y distancia desde el centro. tb coords. 
##df_aemet tiene información del tiempo en días determinados en determinados lugares
##df_municipios tiene información de los municipios y su localización con coords.



In [5]:
df_aemet.head()

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,acebeda-la-id28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,acebeda-la-id28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,Riesgo,0,80,Riesgo,S,20,acebeda-la-id28001


In [6]:
#Limpiamos dfs
df_aemet[["localizacion","id_aemet"]] = df_aemet["localizacion_id"].str.rsplit("-",n=1,expand= True)
df_aemet.drop(columns = "localizacion_id",inplace= True)
df_municipios = df_municipios.T.rename(columns = {0:"Latitud",1:"Longitud",2:"Localidad"})
df_municipios.reset_index(inplace=True)

# Transformar "index", "municipio", "localización" a lista y hacer un diccionario para que todo sea igual

lista_municipios_aemet = list (df_aemet["localizacion"].unique())
lista_municipios_4sq = list(df_4sq["municipio"].unique())
lista_index_municipios = list(df_municipios["index"])

# Hacer un isin para ver que todos los municipios coinciden
#lista 4sq están todos en municipios
#{'cabrera-la', 'leganes', 'molinos-los'} no estan en municipios
set(lista_municipios_aemet).difference(set(lista_index_municipios))

# Filtrar las filas donde 'localizacion' es 'Cabrera-la', 'Leganes' o 'Molinos-los'
filtro = df_aemet.query('localizacion in ["cabrera-la", "leganes", "molinos-los"]')
df_aemet = df_aemet.drop(filtro.index)

#Examinamos valores extraños en df_aemet. La documentación oficial indica que el término "Riesgo" no es adecuado, pues se refiere a un término no deseado. Lo dropeamos
df_aemet["prob._precip.__(%)"].describe()
df_aemet["prob._precip.__(%)"].value_counts()

## Renombrar la columna actual
df_aemet = df_aemet.rename(columns={"prob._precip.__(%)": "Prob_precipitacion"})


##Drop Riesgo
filtro_riesgos = df_aemet.query('Prob_precipitacion in ["Riesgo"]')
df_aemet = df_aemet.drop(filtro_riesgos.index)

#Dropear avisos pues no aportan información (No sabemos de qué avisan)
df_aemet.drop(columns = ["avisos"],inplace = True)

#Relocalizamos IDAEMET al principio 
idaemet_col = df_aemet.pop('id_aemet')
df_aemet.insert(0, "id_aemet", idaemet_col)
df_aemet.columns
#df_aemet ok :)


df_municipios.reset_index(inplace=True)
df_municipios.rename(columns = {"level_0": "denominador"},inplace= True)
#df_municipios ok

# Convertir las columnas 'index' y 'denominador' en un diccionario
diccionario_anadir = df_municipios.set_index('index')['denominador'].to_dict()

# Aplicar el mapeo en la columna 'localizacion' de df_aemet
df_aemet["denominado"] = df_aemet["localizacion"].map(diccionario_anadir)
df_4sq["denominado"] = df_4sq["municipio"].map(diccionario_anadir)

#Nuevas columnas para establecer FKs 

In [7]:
df_4sq

,municipio,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,denominado
0,ajalvir,Park,4d3c6c5b84d46ea87dddfd5c,LikelyOpen,563.0,/v3/places/4d3c6c5b84d46ea87dddfd5c,Diverjungla,"Calle Segovia, 20 (Pol. Ind. Donada), 28864 Aj...",40.529921,-3.484341,1
1,ajalvir,Park,4f54f7bfe4b036244d02685b,VeryLikelyOpen,1724.0,/v3/places/4f54f7bfe4b036244d02685b,Parque Infantil,NaN,40.543867,-3.464773,1
2,ajalvir,Park,7d4ef99ca2dd4789f032a90c,LikelyOpen,333.0,/v3/places/7d4ef99ca2dd4789f032a90c,Chiquitin,Ajalvir Madrid,40.531662,-3.482835,1
3,alamo-el,Park,53de1144498e8fd8d8cb3537,LikelyOpen,118.0,/v3/places/53de1144498e8fd8d8cb3537,Isla Perejil,28607 El Alamo Comunidad de Madrid,40.230993,-3.991942,3
4,alamo-el,Park,51a8f451498ecb8334de37df,LikelyOpen,582.0,/v3/places/51a8f451498ecb8334de37df,Parque de Madera,NaN,40.234636,-3.995166,3
...,...,...,...,...,...,...,...,...,...,...,...
902,villaviciosa-de-odon,Park,51ffcb685019c1bb402374fd,LikelyOpen,1043.0,/v3/places/51ffcb685019c1bb402374fd,Matadero de Villaviciosa CQB ARISOFT,28670 Villaviciosa de Odón Comunidad de Madrid,40.351365,-3.890769,173
903,villaviciosa-de-odon,Park,4c7a6f88566db60c43e7470e,LikelyOpen,1626.0,/v3/places/4c7a6f88566db60c43e7470e,Parque del Castillo,"Avenida del Castillo, 28670 Villaviciosa de Od...",40.357793,-3.881113,173
904,villaviciosa-de-odon,Monument,5834bfa364f54721bc08814d,Unsure,747.0,/v3/places/5834bfa364f54721bc08814d,Discoteca Doblón,"Avenida Principe de Asturias, 94, Villaviciosa...",40.360011,-3.908360,173
905,zarzalejo,Park,4ed124af5c5c9528fa2230de,LikelyOpen,1577.0,/v3/places/4ed124af5c5c9528fa2230de,Pinar Zarzalejo,Pinar,40.540691,-4.165816,175


In [8]:
diccionario_anadir

{'acebeda-la': 0,
 'ajalvir': 1,
 'alameda-del-valle': 2,
 'alamo-el': 3,
 'alcala-de-henares': 4,
 'alcobendas': 5,
 'alcorcon': 6,
 'aldea-del-fresno': 7,
 'algete': 8,
 'alpedrete': 9,
 'ambite': 10,
 'anchuelo': 11,
 'aranjuez': 12,
 'arganda-del-rey': 13,
 'arroyomolinos': 14,
 'atazar-el': 15,
 'batres': 16,
 'becerril-de-la-sierra': 17,
 'belmonte-de-tajo': 18,
 'berrueco-el': 19,
 'berzosa-del-lozoya': 20,
 'boadilla-del-monte': 21,
 'boalo-el': 22,
 'braojos': 23,
 'brea-de-tajo': 24,
 'brunete': 25,
 'buitrago-del-lozoya': 26,
 'bustarviejo': 27,
 'cabanillas-de-la-sierra': 28,
 'cadalso-de-los-vidrios': 29,
 'camarma-de-esteruelas': 30,
 'campo-real': 31,
 'canencia': 32,
 'carabana': 33,
 'casarrubuelos': 34,
 'cenicientos': 35,
 'cercedilla': 36,
 'cervera-de-buitrago': 37,
 'chapineria': 38,
 'chinchon': 39,
 'ciempozuelos': 40,
 'cobena': 41,
 'collado-mediano': 42,
 'collado-villalba': 43,
 'colmenar-del-arroyo': 44,
 'colmenar-de-oreja': 45,
 'colmenarejo': 46,
 'colme

In [9]:
#Nos conectamos a postgres
try:
    conexion = psycopg2.connect(
        database = "Lab3M4",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432"

)
except OperationalError:
    print("la contraseña es errónea")



In [10]:
# Conexión a la base de datos
try:

    # Crear un cursor
    cursor = conexion.cursor()

    # Crear la tabla municipios sin UNIQUE en index
    query_table_municipios = """
    DROP TABLE IF EXISTS municipios CASCADE;

    CREATE TABLE IF NOT EXISTS municipios (
        denominador INT PRIMARY KEY,
        index VARCHAR(100),
        latitud DECIMAL,
        longitud DECIMAL,
        localidad VARCHAR(100)
    );
    """

    # Crear la tabla aemet referenciando el ID de municipios
    query_table_aemet = """
    DROP TABLE IF EXISTS aemet;

    CREATE TABLE IF NOT EXISTS aemet (
        id SERIAL PRIMARY KEY,
        id_aemet VARCHAR(50), 
        fecha TIMESTAMP,
        cielo VARCHAR(50),
        temp_c INT,
        sen_termica_c INT,
        racha_max_kmh DECIMAL,
        precipitacion_mm DECIMAL,
        nieve_mm INT,
        humedad_relativa INT,
        prob_precipitacion INT,
        prob_nieve INT,
        prob_tormenta INT,
        direccion_viento VARCHAR(50),
        velocidad_viento INT,
        denominado INT,
        localizacion VARCHAR (50),
            FOREIGN KEY (denominado) 
            REFERENCES municipios(denominador)
            ON UPDATE CASCADE
            ON DELETE CASCADE
    );
    """

    # Crear la tabla foursquare referenciando el ID de municipios
    query_table_4sq = """
    DROP TABLE IF EXISTS foursquare;

    CREATE TABLE IF NOT EXISTS foursquare (
        id SERIAL PRIMARY KEY,
    
        municipio VARCHAR(50),
        category VARCHAR(100),
        fsq_id VARCHAR(100),
        closed_bucket VARCHAR(50),
        distance DECIMAL,
        link TEXT,
        name VARCHAR(100),
        address TEXT,
        latitude DECIMAL,
        longitude DECIMAL,
        CONSTRAINT fk_municipio_foursquare
            FOREIGN KEY (denominado) 
            REFERENCES municipios(denominador)
            ON UPDATE CASCADE
            ON DELETE CASCADE
    );
    """

    # Crear las tablas
    cursor.execute(query_table_municipios)
    conexion.commit()

    cursor.execute(query_table_aemet)
    conexion.commit()

    cursor.execute(query_table_4sq)
    conexion.commit()

    # Cerrar el cursor y la conexión
    cursor.close()
    conexion.close()

except Exception as e:
    print(f"Ocurrió un error: {e}")
    if conexion:
        conexion.rollback()


Ocurrió un error: no existe la columna «denominado» referida en la llave foránea



In [11]:
def insertar_datos(df, insert_query):
    for row in df.itertuples(index=False, name=None):
        cursor.execute(insert_query, row)


In [12]:
len(df_aemet.columns)

16

In [13]:

# Intentar ejecutar el código y manejar excepciones
try:
    #Nos conectamos a postgres
    try:
        conexion = psycopg2.connect(
            database = "Lab3M4",
            user = "postgres",
            password = "admin",
            host = "localhost",
            port = "5432"

    )
    except OperationalError:
        print("la contraseña es errónea")

    # Crear un cursor
    cursor = conexion.cursor()

    # Insertar datos en la tabla "municipios"
    insert_municipios_query = """
        INSERT INTO municipios (denominador, index, latitud, longitud, localidad)
        VALUES (%s, %s, %s, %s, %s);
    """
    insertar_datos(df_municipios, insert_municipios_query)

    # Insertar datos en la tabla "aemet"
    insert_aemet_query = """
        INSERT INTO aemet (
            id_aemet,fecha, cielo, temp_c, sen_termica_c, racha_max_kmh, precipitacion_mm, nieve_mm, 
            humedad_relativa, prob_precipitacion, prob_nieve, prob_tormenta, direccion_viento, 
            velocidad_viento, localizacion, denominado
        ) VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    insertar_datos(df_aemet, insert_aemet_query)

    # Insertar datos en la tabla "foursquare"
    insert_foursquare_query = """
        INSERT INTO foursquare (
            municipio, category, fsq_id, closed_bucket, distance, link, name, address, latitude, longitude,denominado
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s);
    """
    insertar_datos(df_4sq, insert_foursquare_query)

    # Confirmar los cambios
    conexion.commit()

    # Cerrar el cursor y la conexión
    cursor.close()
    conexion.close()

except Exception as e:
    print(f"Ocurrió un error: {e}")
    # Deshacer los cambios si ocurre un error
    if conexion:
        conexion.rollback()


In [14]:
df_4sq.columns

Index(['municipio', 'category', 'fsq_id', 'closed_bucket', 'distance', 'link',
       'name', 'address', 'latitude', 'longitude', 'denominado'],
      dtype='object')

In [15]:
# #Guardamos como csv para usarlos en Dbeaver
# df_aemet.to_csv("df_aemet_bdd.csv", sep = ',', index = False, encoding= "utf-8")
# df_municipios.to_csv("df_municipios_bdd.csv", sep = ',',index = False, encoding= "utf-8" )
# df_4sq.to_csv("df_4sq.csv", sep = ',',index = False, encoding= "utf-8" )